In [ ]:
%matplotlib widget
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shutil
import copy
import datetime
import re
import string
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.colors as colors
from pathlib import Path

# from scipy import ndimage as nd

import flopy
print("flopy path: {}".format(flopy.__path__))
print("flopy version: {}".format(flopy.__version__))
import pyemu

import sys
sys.path.append("..")
from scripts import utils
from scripts import build

# Steady-State "history period" model build
The initial step in this notebook reads in the existing MODFLOW-2005 model built using groundwater vistas (Rekker, 2012), and re-builds a MODFLOW-NWT model using the script located `../scripts/build.py`. The 'build_from_orig' function in the build.py script performs this task

In [ ]:
base_m = build.build_from_orig(version='ss', run=True)

# PEST interface setup
This notebook makes extensive use of the `PstFrom` functionality in `pyemu` to set up multipliers on parameters (function defined in `build.py` module).

Observations are also defined, assigned initial values, and weights based on preliminary assumptions about error.


In [ ]:
nreals = 300 # number of realisations in ensemble
pst = build.setup_pst(base_m, nreals=nreals) # set-up PEST interface

#### using `pestpp-ies`, settting `noptmax=-1` here we run the history period prior Monte Carlo analysis
* using the number of realizations specified by `nreals`
* will run in parallel locally using the number of cores specified by `num_workers` in `../scripts/build.py`
* creates a new directory called `master_hist_prior/` that will contain the PEST++ output from the parallel Monte Carlo
* upon running will generate worker directories

In [ ]:
# either use base_m model object as defined above or explicitly pass str:
m_d = "master_hist_prior"
noptmax=-1
pst = "Dunedin_SS_base.pst"
utils.prep_and_run(pst, t_d="template_hist_ss", m_d=m_d, nreals=nreals, noptmax=noptmax)

#### We load in the prior ensemble of simulated outputs

In [ ]:
# read modflow model
m_d = "master_hist_prior"
m_d =os.path.join("..", m_d)
m = flopy.modflow.Modflow.load(
    f="SouthDun_SS.nam", 
    model_ws=m_d, 
    version='mfnwt',
    exe_name='mfnwt.exe', 
    verbose=False, 
    check=True
)
# PROCESS SWEEP OUTPUTS From Prior MC
# re-read pst control file from master dir
try:
    pst.filename = Path(pst.filename)
    pst = pyemu.Pst(os.path.join(m_d, pst.filename.name))
except (AttributeError, NameError):
    pst = "Dunedin_SS_base.pst"
    pst = pyemu.Pst(os.path.join(m_d, pst))
# get pest obs data from pest control file object
obs = pst.observation_data

# pulling pest file name from pst object
pstfnme = Path(pst.filename)
# show initial phi componenets
print("Group phi components from base model run")
display(pd.DataFrame.from_dict(pst.phi_components, orient='index'))
# re ensemble outputs
oe_pr = utils.try_load_ensemble(pst, os.path.join(m_d, f"{pstfnme.stem}.0.obs.csv"), kind='obs')
# read obs + noise ensemble (IES drew this from obs weights when we ran prior)
# revist weight on water level obs
nnzobs = obs.loc[pst.nnz_obs_names]
wlobs = nnzobs.loc[nnzobs.oname=="wl"].index

#### Pull out the water level simulated outputs and process probabilities

In [ ]:
hdobs = obs.loc[obs.oname == "hd"].astype({c: int for c in ['kper', 'k', 'i', 'j']})
hdobs['top'] = m.dis.top.array[tuple(hdobs[['i', 'j']].T.values)]
hdvtop = oe_pr.T.loc[hdobs.index].sub(hdobs.top, axis=0)
# calc probabilities of exceed for every output
# Transpose obs ensemble (ensemble outputs), slice for just hd obs,
# substract model top from every realisation,
# where positive simulated head exceeds model top, count reals where positive, divide by nreal
hdobs['prob'] = (hdvtop > 0).sum(axis=1) / oe_pr.shape[0]

# create an array from these obs -- WILL NEED TO BE DIF IF MULTPLE KPER AND LAYERS
ar_prior = np.zeros((m.nrow, m.ncol))  # blank array

# add elements from dataframe
ar_prior[tuple(hdobs[['i', 'j']].T.values)] = hdobs.prob

fig, ax = plt.subplots(1, 1, figsize=(6, 6))
im = ax.imshow(np.ma.masked_where(m.bas6.ibound.array[0] == 0, ar_prior * 100),
               cmap="jet", interpolation='none')
ax.set_title('Prior probability of groundwater inundation', fontsize='12')
fig.colorbar(im)
fig.tight_layout()

#### We plot the prior distribution of the total drain flux -versus- the estimated observation + error

In [ ]:
# Plotting ensemble output histograms
#load obs+noise ensemble
obsplus = utils.try_load_ensemble(pst, os.path.join(m_d, f"{pstfnme.stem}.obs+noise.csv"), kind='obs')
# just slice out drain obs
drnsumobs = obs.loc[obs.oname=="drnsum"]#.astype({c:int for c in ['kper','k','i','j']})
# Prior sim out of drain sum ob
drnsumoe_pr = oe_pr.loc[:, drnsumobs.index]
# obs plus noise for output 
dnobsplus = obsplus.loc[:, drnsumobs.index]
# plot histos
fig, ax = plt.subplots(1,1, figsize=(6,4))
_ = drnsumoe_pr.hist(ax=ax, bins=25, color='0.5', alpha=0.5, density=False)
_ = dnobsplus.hist(ax=ax, bins=25, color='r', alpha=0.5, density=False)
ax.set_title(label='Storm / wastewater flux (prior -vs- obs)', fontsize=7)
ax.tick_params(axis='x', labelsize=7)
ax.tick_params(axis='y', labelsize=7)
ax.set_xlabel('Flux m$^3$/day', fontsize=7)
ax.set_ylabel('Num. reals', fontsize=7)

#### We plot the prior distribution of groundwater levels -versus- all water level observations + the estimated error

In [ ]:
# Prior simulatout outputs for WL obs vs obs+noise reals
w_obs = obs.loc[obs.index.str.contains('sitename') & (obs.weight>0)].astype({'i':int, 'j':int})

for ob in w_obs.index:
    top = m.dis.top.array[(w_obs.loc[ob].i, w_obs.loc[ob].j)]
    # Prior sim out
    tp_pr = oe_pr.loc[:, ob]
    # OBS 
    # obsplus = pd.read_csv(os.path.join(m_d, "Dunedin_SS_base.obs+noise.csv"), index_col=0)
    tpobsplus = obsplus.loc[:, ob]
    
    fig, ax = plt.subplots(1,1, figsize=(6,4))
    tp_pr.hist(ax=ax, bins=25, color='0.5', alpha=0.5, density=False)
    tpobsplus.hist(ax=ax, bins=25, color='r', alpha=0.5, density=False)
    ys = ax.get_ylim()
    ax.plot((top, top), ys, color='k', ls='--')
    ax.set_title(ob, fontsize=6)
    ax.tick_params(axis='x', labelsize=7)
    ax.tick_params(axis='y', labelsize=7)
    ax.set_xlabel('Groundwater level m (OMD)', fontsize=7)
    ax.set_ylabel('Num. reals', fontsize=7)

#### We de-weight groundwater level observations located in e.g., the perched dune aquifer system (before history matching)

In [ ]:
# first increase variance on highly uncertain drain obs estimate
obs.loc[drnsumobs.index, 'weight'] = 1/500
# drop moana rua shallow
mrs = w_obs.loc[w_obs.sitename=='moana-rua-shallow'] 
obs.loc[mrs.index, 'weight'] = 0
# drop Tonga Park deep
tpd = w_obs.loc[w_obs.sitename=='tonga-park-deep'] 
obs.loc[tpd.index, 'weight'] = 0
# reweight fitzroy street
ftz = w_obs.loc[w_obs.sitename=='fitzroy-st'] 
obs.loc[ftz.index, 'weight'] = 0
# reweight Turukina Rd
trd = w_obs.loc[w_obs.sitename=='turakina-rd'] 
obs.loc[trd.index, 'weight'] = 0
# reweight Scout hall shallow
shs = w_obs.loc[w_obs.sitename=='scout-hall-shallow'] 
obs.loc[shs.index, 'weight'] = 0
# reweight OMES
oms = w_obs.loc[w_obs.sitename=='omes'] 
obs.loc[oms.index, 'weight'] = 0
# reweight holiday-park
hyd = w_obs.loc[w_obs.sitename=='holiday-park'] 
obs.loc[hyd.index, 'weight'] = 0
# reweight Richardson
ric = w_obs.loc[w_obs.sitename=='richardson-st'] 
obs.loc[ric.index, 'weight'] = 0
# reweight ravelston
rav = w_obs.loc[w_obs.sitename=='ravelston'] 
obs.loc[rav.index, 'weight'] = 0
# reweight forbury
fby = w_obs.loc[w_obs.sitename=='forbury-park-raceway'] 
obs.loc[fby.index, 'weight'] = 0

#### We draw the observation plus noise based on observation weights

In [ ]:
# redraw obs plus noise
obsplus = pyemu.ObservationEnsemble.from_gaussian_draw(
    pst, num_reals=nreals, fill=True)
obsplus.add_base()
obsplus = obsplus._df
# force drawn obs reals for drain to be < -500
dnobsplus = obsplus.loc[:, drnsumobs.index]
dnobsplus[dnobsplus > 0] = 0
obsplus.loc[:, drnsumobs.index] = dnobsplus
# for fun plot drain prior with obs+noise reals
fig, ax = plt.subplots(1,1, figsize=(6,4))
_ = drnsumoe_pr.hist(ax=ax, bins=25, color='0.5', alpha=0.5, density=False)
_ = obsplus.loc[:, drnsumobs.index].hist(
    ax=ax, bins=25, color='r', alpha=0.5, density=False)
ax.set_title(label='Storm / wastewater flux (prior -vs- obs)', fontsize=7)
ax.tick_params(axis='x', labelsize=7)
ax.tick_params(axis='y', labelsize=7)
ax.set_xlabel('Flux m$^3$/day', fontsize=7)
ax.set_ylabel('Num. reals', fontsize=7)

# similarly clip drawn wl noise obs to below model surface
w_obs['top'] = m.dis.top.array[tuple(w_obs[['i','j']].values.T)]
wlobsplus = obsplus.loc[:, w_obs.index]
wlobsplus = wlobsplus.where(~wlobsplus.gt(w_obs.top), w_obs.top, axis=1)
obsplus.loc[:, w_obs.index] = wlobsplus
for ob in w_obs.index:
    top = m.dis.top.array[(w_obs.loc[ob].i, w_obs.loc[ob].j)]
    # Prior sim out
    tp_pr = oe_pr.loc[:, ob]
    # OBS 
    # obsplus = pd.read_csv(os.path.join(m_d, "Dunedin_SS_base.obs+noise.csv"), index_col=0)
    tpobsplus = obsplus.loc[:, ob]
    
    fig, ax = plt.subplots(1,1, figsize=(6,4))
    tp_pr.hist(ax=ax, bins=25, color='0.5', alpha=0.5, density=False)
    tpobsplus.hist(ax=ax, bins=25, color='r', alpha=0.5, density=False)
    ys = ax.get_ylim()
    ax.plot((top, top), ys, color='k', ls='--')
    ax.set_title(ob, fontsize=6)
    ax.tick_params(axis='x', labelsize=7)
    ax.tick_params(axis='y', labelsize=7)
    ax.set_xlabel('Groundwater level m (OMD)', fontsize=7)
    ax.set_ylabel('Num. reals', fontsize=7)

#### Balancing objective function

In [ ]:
# pdc work
# set less_than obsplus to be equal to obs val (as we are determining them to be model top always)
lessthanobs = nnzobs.loc[nnzobs.obgnme.str.startswith('less_')]
# modify obs+noise to remove noise from less than obs
obsplus.loc[:, lessthanobs.index] = lessthanobs.obsval.values
#display(obsplus.loc[:, lessthanobs.index].head())
# write modified obs+noise ensemble to prior master and template
# pass back to an obsensemble object
obsplus = pyemu.ObservationEnsemble(pst, obsplus)
obsplus.to_binary(os.path.join(m_d, "Dunedin_SS_base_rw.obs+noise.jcb"))
obsplus.to_binary(os.path.join("..", "template_hist_ss", "Dunedin_SS_base_rw.obs+noise.jcb"))

# min mean and max of simulated outputs (used to determine conflicts)
nnzobs['minout'] = oe_pr.loc[:, nnzobs.index].min()
nnzobs['meanout'] = oe_pr.loc[:, nnzobs.index].mean()
nnzobs['maxout'] = oe_pr.loc[:, nnzobs.index].max()

# min mean and max of obs+noise reals (used to determine conflicts)
nnzobs['obsmin'] = obsplus.loc[:, nnzobs.index].min()
nnzobs['obsmean'] = obsplus.loc[:, nnzobs.index].mean()
nnzobs['obsmax'] = obsplus.loc[:, nnzobs.index].max()

# need to treat less than obs and normal obs a bit different
lessthanobs = nnzobs.loc[lessthanobs.index] # only conflict where simmin>obsmax
normalobs = nnzobs.loc[nnzobs.index.difference(lessthanobs.index)] # also conflict where simmax<obsmin

# pull out conflicting obs
conflict_obs = pd.concat(
    [normalobs.loc[normalobs.minout > normalobs.obsmax,:],
     normalobs.loc[normalobs.maxout < normalobs.obsmin,:],
     lessthanobs.loc[lessthanobs.minout > lessthanobs.obsmax,:]]
)
#display(conflict_obs)
#display(conflict_obs.groupby('obgnme').count())
#display(obs.loc[conflict_obs.index, ['weight']])

# set weight of conflicting obs to zero
obs.loc[conflict_obs.index, 'weight'] = 0
#display(obs.loc[conflict_obs.index, ['weight']])
pst.observation_data = obs

# Weight Balancing steps
# make sure we are using the calculated residual from the prior to provide the
# current phi components by group
# generate residual dataframe from ensemble
# (will use base realistion or mean of ensemble)
if "real_name" in oe_pr.columns:
    oe_pr = oe_pr.set_index('real_name')

# try with prior mean (recalcs phi comps)
res = oe_pr.mean().to_frame()
res.columns = ['modelled']
res['measured'] = obsplus.mean()
res['group'] = obs.obgnme
# below methos expeting "name" column
res['name'] = res.index
pst.set_res(res)

# build dict of desired group weights
phi_comps = pst.phi_components
#display("\nPHI COMPONENTS BEFORE BALANANCING")
#display(pd.DataFrame.from_dict(phi_comps, orient='index'))

# only want to work with non zero contributing groups
phi_comps = {k: phi_comps[k] for k in phi_comps.keys() if phi_comps[k] > 0}
#display(pd.DataFrame.from_dict(phi_comps, orient='index'))
maxgpcont = pd.DataFrame.from_dict(phi_comps, orient='index').max()[0]

obs.loc[pst.nnz_obs_names].groupby('obgnme')['obgnme'].count()
#display("\nattempting to balance weights") #  so less-thans contribute more (as more obs)")

new_phi_comps = pd.DataFrame.from_dict(phi_comps, orient='index')
new_phi_comps.loc[:, :] = maxgpcont  # set groups to same value
# reduce relative on inequality and drain
new_phi_comps.loc[new_phi_comps.index == "less_hd", :] *= 1.0e-1
new_phi_comps.loc[(new_phi_comps.index ==
                   "oname:drnsum_otype:lst_usecol:sum"), :] *= 1.0e-1
#display("\nDESIRED PHI COMPONENTS AFTER BALANCING")
#display(new_phi_comps)

# send desired group weights to pyemu method of adjusting group wise
pst.adjust_weights(obsgrp_dict=new_phi_comps[0].to_dict())
display("\nPHI COMPONENTS AFTER BALANANCING")
#print(pst.observation_data.loc[pst.nnz_obs_names])
display(pd.DataFrame.from_dict(pst.phi_components, orient='index'))

#### Here we prepare for, and run history matching

In [ ]:
pst.pestpp_options["ies_obs_en"] = "Dunedin_SS_base_rw.obs+noise.jcb"
pst.pestpp_options["ies_restart_observation_ensemble"] = "Dunedin_SS_base_rw.0.obs.jcb"
pst.pestpp_options["ies_par_en"] = "Dunedin_SS_base.0.par.jcb"
pst.pestpp_options["ies_subset_size"] = int(0.1 * nreals)
# 10% of nreals (later version of PESTPP can just use -10)
# save update pst control file (in master prior for now)
if os.path.exists(
        os.path.join("..", "template_hist_ss", "Dunedin_SS_base.pst.bckup")
):
    shutil.copy(os.path.join("..", "template_hist_ss", "Dunedin_SS_base.pst.bckup"),
                os.path.join("..", "template_hist_ss", "Dunedin_SS_base.pst"))
else:
    shutil.copy(os.path.join("..", "template_hist_ss", "Dunedin_SS_base.pst"),
                os.path.join("..", "template_hist_ss", "Dunedin_SS_base.pst.bckup"))
pst.write(os.path.join(m_d, "Dunedin_SS_base_rw.pst"), version=2)
pst.write(os.path.join("..", "template_hist_ss", "Dunedin_SS_base.pst"), version=2)
# copy files that we need in template
shutil.copy(os.path.join(m_d, "Dunedin_SS_base.0.obs.jcb"),
            os.path.join("..", "template_hist_ss", "Dunedin_SS_base_rw.0.obs.jcb"))
shutil.copy(os.path.join(m_d, "Dunedin_SS_base.0.par.jcb"),
            os.path.join("..", "template_hist_ss", "Dunedin_SS_base.0.par.jcb"))

#### using `pestpp-ies`, settting `noptmax=6` here we history match to gw levels obs, total drain flux and inequality constraints
* using the number of realizations specified by `nreals`
* will run in parallel locally using the number of cores specified by `num_workers` in `../scripts/build.py`
* creates a new directory called `master/` that will contain the PEST++ output from each iteration of history matching
* upon running will generate worker directories

In [ ]:
# either use base_m model object as defined above or explicitly pass str:
noptmax=6 # number of pestpp-ies iterations
num_workers = 15 # number of parallel runs
pstfile = "Dunedin_SS_base.pst" # pst control file
utils.prep_and_run(pstfile, t_d="template_hist_ss", 
                   nreals=nreals, noptmax=noptmax, nworker=num_workers)

#### We process the posterior ensemble outputs following history matching
#### Let's plot the posterior probability of groundwater inundation

In [ ]:
# new master directory
m_d = os.path.join("..", "master")

# pestpp-ies iteration
it = noptmax

# load posterior ensemble
oe_po = utils.try_load_ensemble(pst, os.path.join(m_d, f"Dunedin_SS_base.{it}.obs.jcb"), 'obs')

# get pest obs data
obs = pst.observation_data

# just hd outputs
hdobs = obs.loc[obs.obgnme == "oname:hd_otype:lst_usecol:obsval"].astype({c:int for c in ['kper','k','i','j']})

# add column that aligns model top info to hd output names
hdobs['top'] = m.dis.top.array[tuple(hdobs[['i','j']].T.values)]

# calc probabilities of exceed for every output
# Transpose obs ensemble (ensemble outputs), slice for just hd obs, 
# substract model top from every realisation,
# where positive simulated head exceeds model top, count reals where positive, divide by nreal
hdobs['prob'] = (oe_po.T.loc[hdobs.index].sub(hdobs.top, axis=0) > 0).sum(axis=1)/oe_po.shape[0]

# create an array from these obs -- WILL NEED TO BE DIF IF MULTPLE KPER AND LAYERS
ar_post = np.zeros((m.nrow, m.ncol)) # blank array

# add elements from dataframe
ar_post[tuple(hdobs[['i','j']].T.values)] = hdobs.prob
plt.figure(figsize=(6,6))
plt.imshow(np.ma.masked_where(m.bas6.ibound.array[0]==0, ar_post), cmap="jet")
plt.title('Posterior probability of groundwater inundation')
plt.colorbar()

#### We plot the prior versus posterior drain flux distributions

In [ ]:
drnsumobs = obs.loc[obs.index.str.startswith('oname:drnsum')]#.astype({c:int for c in ['kper','k','i','j']})
# Prior sim out
drnsumoe_pr = oe_pr.loc[:, drnsumobs.index]
# Posterior sim out
drnsumoe_po = oe_po.loc[:, drnsumobs.index]

# OBS
obsplus = pyemu.ObservationEnsemble.from_binary(
    pst, os.path.join(m_d, "Dunedin_SS_base_rw.obs+noise.jcb"))
# obsplus = pd.read_csv(os.path.join(m_d, "Dunedin_SS_base.obs+noise.csv"), 
#                       index_col=0)
dnobsplus = obsplus.loc[:, drnsumobs.index]

fig, ax = plt.subplots(1,1, figsize=(6,4))
drnsumoe_pr.hist(ax=ax, bins=25, color='0.5', alpha=0.5, density=False)
drnsumoe_po.hist(ax=ax, bins=25, color='b', alpha=0.5, density=False)
dnobsplus.hist(ax=ax, bins=25, color='r', alpha=0.5, density=False)
ax.set_title(label='Storm / wastewater flux (prior -vs- post. -vs- obs)', fontsize=7)
ax.tick_params(axis='x', labelsize=7)
ax.tick_params(axis='y', labelsize=7)
ax.set_xlabel('Flux m$^3$/day', fontsize=7)
ax.set_ylabel('Num. reals', fontsize=7)
ax.set_xlim(-6000, 1000)
#ax.set_ylim(0, 50)
#plt.savefig("plots/prior_drn_flux")

#### We plot the prior versus posterior ensemble outputs for the gw level observation locations

In [ ]:
w_obs = obs.loc[
    obs.index.str.contains('sitename') & (obs.kper=='0')
].astype({'i':int, 'j':int})

m.dis.top.array[(w_obs.loc[ob].i, w_obs.loc[ob].j)]

for ob in w_obs.index:
    top = m.dis.top.array[(w_obs.loc[ob].i, w_obs.loc[ob].j)]
    # Prior sim out
    tp_pr = oe_pr.loc[:, ob]
    # Posterior sim out
    tp_po = oe_po.loc[:, ob]
    # OBS 
    # obsplus = pd.read_csv(os.path.join(m_d, "Dunedin_SS_base.obs+noise.csv"), index_col=0)
    tpobsplus = obsplus.loc[:, ob]
    
    fig, ax = plt.subplots(1,1, figsize=(6,4))
    tp_pr.hist(ax=ax, bins=25, color='0.5', alpha=0.5, density=False)
    tp_po.hist(ax=ax, bins=25, color='b', alpha=0.5, density=False)
    tpobsplus.hist(ax=ax, bins=25, color='r', alpha=0.5, density=False)
    ys = ax.get_ylim()
    ax.plot((top, top), ys, c='k', ls='--')
    ax.plot((w_obs.loc[ob, 'obsval'], w_obs.loc[ob, 'obsval']), ys, c='r', ls='--')
    ax.set_title(ob, fontsize=8)
    #ax.set_title(label='Storm / wastewater flux (prior -vs- obs)', fontsize=7)
    ax.tick_params(axis='x', labelsize=7)
    ax.tick_params(axis='y', labelsize=7)
    #ax.set_xlim(98, 103)
    ax.set_xlabel('groundwater level m (OMD)', fontsize=7)
    ax.set_ylabel('Num. reals', fontsize=7)

#### We plot the % reduction in uncertainty for the groundwater level prediction (for the history matching period)

In [ ]:
# re ensemble outputs
#load obs po ensemble using Brioch's try_load_ensemble function
obs_ens_po = utils.try_load_ensemble(pst, os.path.join(m_d, f"Dunedin_SS_base.{it}.obs.jcb"), kind='obs')
obs_reals_po = obs_ens_po.T
#obs_reals
obs_po = obs_reals_po.loc[obs_reals_po.index.str.contains('oname:hd_otype')] #.astype({c:int for c in ['i','j']})

obs_ens_pr = utils.try_load_ensemble(pst, os.path.join(m_d, "Dunedin_SS_base.0.obs.jcb"), kind='obs')
obs_reals_pr= obs_ens_pr.T
#obs_reals
obs_pr = obs_reals_pr.loc[obs_reals_pr.index.str.contains('oname:hd_otype')] #.astype({c:int for c in ['i','j']})
percent = 100 * (1.0 - obs_po.std(axis=1)/obs_pr.std(axis=1))
ar = np.zeros(m.dis.top.shape) * np.nan
ar[tuple(
    pst.observation_data.loc[obs_pr.index][['i', 'j']].astype(int).values.T
)] =  percent # par_obs.loc[:, 0].values

plt.figure()
plt.imshow(np.ma.masked_where(ar<0, ar), cmap = 'plasma', vmin=0, vmax=100)
plt.colorbar()

# Transient projection model build
This step in the notebook reads in the existing MODFLOW-2005 model built using groundwater vistas (Rekker, 2012), and re-builds a transient MODFLOW-NWT model using the script located `../scripts/build.py`. The 'build_from_orig' function in the build.py script performs this task (`version='pred'`)

In [ ]:
pred_m = build.build_from_orig(version='pred', run=True)

# set-up PEST interface for projection model

In [ ]:
pred_pst = build.setup_pst("Dunedin_Pred_base", nreals=nreals)

# Attach SLR scenario (e.g., SSP5)

#### We attach the posterior ensemble (fit=6) and attach SLR scenario using 'attach_scenario' function in build.py script 

In [ ]:
# nreals=300  # number of realisations for predictive ensemble (if less than nreals in historymatching)
fit=6  # iteration number 
nworkers=15  # number of parallel realisation to run (lower if cpu limited)
t_d = "template_proj_101"
scen = "SSP5"
#m_d = f"master_{scen}"
m_d = "master_SSP5"
pred_m = build.attach_scenario(model=t_d, scen=scen) # attach SLR scenario

#### Run projection model posterior Monte Carlo using PEST++ (combine posterior ensemble of parameters for history period with unconditioned temporal parameters)

In [ ]:
utils.prep_and_run("Dunedin_Pred_base.pst", t_d=t_d, m_d=m_d,
                    nreals=nreals, noptmax=-1, nworker=nworkers,
                    prior_pe="prior_pe.jcb",
                    post_pe=os.path.join("..", "master",
                                         f"Dunedin_SS_base.{fit}.par.jcb"))

#### Read and process ensemble outputs for projection period

In [ ]:
#pred_pst = "Dunedin_Pred_base.pst"
m_d = "master_SSP5"
m_d =os.path.join("..", m_d)
m = flopy.modflow.Modflow.load(
    f="SouthDun_100.nam", 
    model_ws=m_d, 
    version='mfnwt',
    exe_name='mfnwt.exe', 
    verbose=False, 
    check=True
)
# PROCESS SWEEP OUTPUTS
# re-read pst control file from master dir
try:
    pred_pst.filename = Path(pred_pst.filename)
    pred_pst = pyemu.Pst(os.path.join(m_d, pred_pst.filename.name))
except (AttributeError, NameError):
    pred_pst = "Dunedin_Pred_base.pst"
    pred_pst = pyemu.Pst(os.path.join(m_d, pred_pst))
pstfnme = Path(pred_pst.filename)

#### Read in projection results

In [ ]:
# re ensemble outputs
oe_po_tr = utils.try_load_ensemble(pred_pst, os.path.join(m_d, "Dunedin_Pred_base.0.obs.jcb"), kind='obs')

#### Let's look at the probability of groundwater inundation for a specific year of the scenario (e.g., 2100)

In [ ]:
# get pest obs data
obs = pred_pst.observation_data

yr = 2110 # specify year of scenario
kper = yr - 2010

obs = obs.loc[obs.index.str.contains(f'kper:{kper}')]

# just hd outputs
hdobs = obs.loc[obs.obgnme == "oname:hd_otype:lst_usecol:obsval"].astype({c:int for c in ['kper','k','i','j']})
# add column that aligns model top info to hd output names
hdobs['top'] = m.dis.top.array[tuple(hdobs[['i','j']].T.values)]

# calc probabilities of exceed for every output
# Transpose obs ensemble (ensemble outputs), slice for just hd obs, 
# substract model top from every realisation,
# where positive simulated head exceeds model top, count reals where positive, divide by nreal
hdobs['prob'] = ((oe_po_tr.T.loc[hdobs.index].sub(hdobs.top, axis=0) + 0) > 0).sum(axis=1)/oe_po_tr.shape[0]

# create an array from these obs -- WILL NEED TO BE DIF IF MULTPLE KPER AND LAYERS
ar = np.zeros((m.nrow, m.ncol)) # blank array

# add elements from dataframe
ar[tuple(hdobs[['i','j']].T.values)] = hdobs.prob

plt.figure(figsize=(6,6))
plt.title(yr, fontsize=12)
plt.imshow(np.ma.masked_where(m.bas6.ibound.array[0]==0, ar), cmap='jet')
plt.colorbar()

#### Plot projection period results for total drain flux

In [ ]:
#get pest obs data
obs = pred_pst.observation_data
drnsumobs = obs.loc[obs.index.str.startswith('oname:drnsum')] #.astype({c:int for c in ['kper','k','i','j']})
# Posterior sim out
drnsumoe_df = oe_po_tr.loc[:, drnsumobs.index]
drnsumoe_df_pos = drnsumoe_df * -1

at_0 = drnsumoe_df.loc[:, 'oname:drnsum_otype:lst_usecol:sum_kper:0']
at_20 = drnsumoe_df.loc[:, 'oname:drnsum_otype:lst_usecol:sum_kper:20']
at_40 = drnsumoe_df.loc[:, 'oname:drnsum_otype:lst_usecol:sum_kper:40']
at_60 = drnsumoe_df.loc[:, 'oname:drnsum_otype:lst_usecol:sum_kper:60']
at_90 = drnsumoe_df.loc[:, 'oname:drnsum_otype:lst_usecol:sum_kper:90']

drn_0 = at_0 * -1
drn_20 = at_20 * -1
drn_40 = at_40 * -1
drn_60 = at_60 * -1
drn_90 = at_90 * -1

diff_20 = at_20 - at_0
diff_40 = at_40 - at_0
diff_60 = at_60 - at_0
diff_90 = at_90 - at_0

fig, axes = plt.subplots(1,2, figsize=(12,4))

ax = axes[0]
drnsumoe_df_pos.loc[:, drnsumobs.astype({'kper':int}).sort_values('kper').index].T.reset_index().plot(ax=ax, legend=False, color='red', alpha=0.2)
ax.set_title("Drain flux versus time", fontsize=10)
ax.tick_params(axis='x', labelsize=10)
ax.tick_params(axis='y', labelsize=10)
ax.set_xlim(0,100)
#ax.set_ylim(-6000, 0)
ax.set_xlabel('year of scenario', fontsize=10)
ax.set_ylabel('flux m$^3$/day', fontsize=10)

ax = axes[1]
drn_90.hist(ax=ax, bins=25, color='r', alpha=0.5, density=False, label='2100')
drn_60.hist(ax=ax, bins=25, color='y', alpha=0.8, density=False, label='2070')
drn_40.hist(ax=ax, bins=25, color='b', alpha=0.8, density=False, label='2050')
drn_20.hist(ax=ax, bins=25, color='0.5', alpha=0.9, density=False, label='2030')
ax.set_title("Drain flux PDF", fontsize=10)
ax.tick_params(axis='x', labelsize=10)
ax.tick_params(axis='y', labelsize=10)
ax.set_xlabel('flux m$^3$/day', fontsize=10)
ax.set_ylabel('frequency', fontsize=10)
legend = ax.legend(loc='best', fontsize=10)
#plt.savefig("plots/SSP5/drain_flux_versus_time.pdf")
#ax.set_ylim(0, 30)

#### Plot projection period results of groundwater levels at observation locations

In [ ]:
# get pest obs data
obs = pred_pst.observation_data
#obs.loc[obs.oname=='wl']
w_obs = obs.loc[obs.index.str.contains('sitename')]
sitegroups = w_obs.groupby("sitename")
for site, w_obs_df in sitegroups:
    top = m.dis.top.array[int(w_obs_df.i[0]), int(w_obs_df.j[0])]
    at0 = w_obs_df.loc[w_obs_df.kper=='0'].index[0] #2010
    at20 = w_obs_df.loc[w_obs_df.kper=='20'].index[0] #2030
    at40 = w_obs_df.loc[w_obs_df.kper=='40'].index[0] #2050
    at70 = w_obs_df.loc[w_obs_df.kper=='60'].index[0] #2070
    at90 = w_obs_df.loc[w_obs_df.kper=='90'].index[0] #2100
    at100 = w_obs_df.loc[w_obs_df.kper=='100'].index[0] #2100
    
    # Posterior sim out
    tp_0 = oe_po_tr.loc[:, at0]
    tp_20 = oe_po_tr.loc[:, at20]  
    tp_40 = oe_po_tr.loc[:, at40]
    #tp_60 = oe_po_tr.loc[:, at60]
    tp_70 = oe_po_tr.loc[:, at70]
    tp_90 = oe_po_tr.loc[:, at90]

    tp_d_40 = tp_40 - tp_0
    tp_d_20 = tp_20 - tp_0
    #tp_d_60 = tp_60 - tp_0
    tp_d_70 = tp_70 - tp_0
    tp_d_90 = tp_90 - tp_0
    
    fig, axes = plt.subplots(1,2, figsize=(10,4))
    ax = axes[0]
    
    tp_d_90.hist(ax=ax, bins=25, color='r', density=False, label='2100', alpha=0.6)
    tp_d_70.hist(ax=ax, bins=25, color='y', density=False, label='2070', alpha=0.8)
    tp_d_40.hist(ax=ax, bins=25, color='b', density=False, label='2050', alpha=0.8)
    tp_d_20.hist(ax=ax, bins=25, color='grey', density=False, label='2030', alpha=0.8)
    #tp_d_70.hist(ax=ax, bins=25, color='k', alpha=0.5, density=False)
    ax.set_title(site, fontsize=9)
    ax.tick_params(axis='x', labelsize=7)
    ax.tick_params(axis='y', labelsize=7)
    ax.set_xlabel('gw level change (m)', fontsize=9)
    ax.set_ylabel('frequency', fontsize=9)
    ys = ax.get_ylim()
    ax.plot((0.25,0.25), ys, color='k', ls='--', label='Model top')
    ax.plot((0.25,0.25), ys, color='r', ls='--', label='Decision threshold')
    legend = ax.legend(loc='best', fontsize=7)
                  
    ax = axes[1]            
    #oe_rcp26.loc[:, w_obs_df.astype({'kper':int}).sort_values('kper').index].T.reset_index().plot(ax=ax, legend=False, color='green', alpha=0.2)
    oe_po_tr.loc[:, w_obs_df.astype({'kper':int}).sort_values('kper').index].T.reset_index().plot(ax=ax, legend=False, color='r', alpha=0.2)
    #oe_rcp85.loc[:, w_obs_df.astype({'kper':int}).sort_values('kper').index].T.reset_index().plot(ax=ax, legend=False, color='red', alpha=0.2)
    ax.tick_params(axis='x', labelsize=7)
    ax.tick_params(axis='y', labelsize=7)
    ax.set_xlabel('Years from start of projection', fontsize=9)
    ax.set_ylabel('gw level (m OMD)', fontsize=9)
    ax.set_title(site, fontsize=9)
    xs = ax.get_xlim()   
    ax.plot(xs, (top,top), color='k', ls='--', label='model top')
    ax.set_xlim(0,100)
        
    #plt.savefig("plots/SSP5/SSP5_po_{0}.png".format(site))